In [ ]:
import time

from ipywidgets import Image
from ipywidgets import ColorPicker, IntSlider, link, AppLayout, HBox
from ipycanvas import Canvas, RoughCanvas, hold_canvas

from src import *

## Curvature based segmentation

In [ ]:
width = 600
height = 300

canvas = Canvas(width=width, height=height)
canvas.fill_style = "#c6574f"
canvas.layout.border_width = 3
canvas.layout.border = "solid"
canvas.layout.border_color = "#000000"

drawing = False
position = None
shape = []
c_list = []
a_list = []
t_list = []


def on_mouse_down(x, y):
    global drawing
    global position
    global shape
    global t_list

    drawing = True
    position = (x, y)
    shape = [position]
    t_list = [time.time_ns()]


def on_mouse_move(x, y):
    global drawing
    global position
    global shape
    global t_list

    if not drawing:
        return

    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)

        position = (x, y)

    shape.append(position)
    t_list.append(time.time_ns())


def on_mouse_up(x, y):
    global drawing
    global position
    global shape
    global c_list
    global a_list

    drawing = False
    
    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)

    _, c_list, a_list, seg_pts = findSegmentationPoints(shape)
    for seg_pt in seg_pts:
        prev_stroke_style = canvas.stroke_style
        canvas.stroke_style = "red"
        canvas.fill_circle(x=seg_pt[0], y=seg_pt[1], radius=5)
        canvas.stroke_style = prev_stroke_style


canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

canvas.stroke_style = "#000000"

canvas

In [ ]:
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

fig = px.line(c_list)
fig.update_layout(
    xaxis_title = "Index",
    yaxis_title = "Curvature",
    showlegend=False
)
fig.show()

In [ ]:
fig = px.line(a_list)
fig.update_layout(
    xaxis_title = "Index",
    yaxis_title = "Abnormality",
    showlegend=False
)
fig.show()

## Speed based segmentation

In [ ]:
width = 600
height = 300

canvas = Canvas(width=width, height=height)
canvas.fill_style = "#c6574f"
canvas.layout.border_width = 3
canvas.layout.border = "solid"
canvas.layout.border_color = "#000000"

drawing = False
position = None
shape = []
c_list = []
a_list = []
t_list = []
s_list = []
seg_pts = []


def on_mouse_down(x, y):
    global drawing
    global position
    global shape
    global t_list

    drawing = True
    position = (x, y)
    shape = [position]
    t_list = [time.time_ns()]


def on_mouse_move(x, y):
    global drawing
    global position
    global shape
    global t_list

    if not drawing:
        return

    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)

        position = (x, y)

    shape.append(position)
    t_list.append(time.time_ns())


def on_mouse_up(x, y):
    global drawing
    global position
    global shape
    global t_list
    global s_list
    global seg_pts

    drawing = False
    
    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)

    s_list, seg_pts = findSegmentationPointsSpeedBased(shape, t_list, threshold=100)
    for seg_pt in seg_pts:
        prev_stroke_style = canvas.stroke_style
        canvas.stroke_style = "red"
        canvas.fill_circle(x=seg_pt[0], y=seg_pt[1], radius=5)
        canvas.stroke_style = prev_stroke_style


canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

canvas.stroke_style = "#000000"

canvas

In [ ]:
fig = px.line(s_list)
fig.update_layout(
    xaxis_title = "Index",
    yaxis_title = "Speed",
    showlegend=False
)
fig.add_hline(y=100)
fig.show()

## Execution time comparison

In [ ]:
%timeit findSegmentationPointsSpeedBased(shape, t_list, threshold=50)

In [ ]:
%timeit findSegmentationPoints(shape)